In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
import glob
import os
import tensorflow_decision_forests as tfdf
import sklearn.metrics as sk
import tensorflow_datasets as tfds
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
import os
import random
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 